# Postprocess wrfout file Tutorial

This tutorial will walk through postprocessing forecast data from your own WRF forecast model data using the wrfcast.py module within pvlib.

This tutorial has been tested against the following package versions:
* Python 
* IPython 
* pandas 
* matplotlib 
* netcdf4 1.4.2 

It should work with other Python and Pandas versions. It requires pvlib >= 0.3.0 and IPython >= 3.0.

Authors:
* Jeffrey Sward (jas983@cornell.edu), Cornell University, November 2019

In [4]:
%matplotlib inline
import matplotlib.pyplot as plt

# built in python modules
import datetime
import os
import inspect
import sys

# python add-ons
import numpy as np
import pandas as pd
import xarray as xr
import netCDF4
import wrf

# Import the pvlib module
if sys.platform == 'linux':
    sys.path.append('/home/jsward/Documents/01_Research/01_Renewable_Analysis/WRF/pvlib-python')
import pvlib
from pvlib.wrfcast import WRF
from optwrf.util import _wrf2xarray

In [2]:
# Find the absolute file path to your pvlib installation
pvlib_abspath = os.path.dirname(os.path.abspath(inspect.getfile(pvlib)))

# absolute path to WRF data file
datapath = os.path.join(pvlib_abspath, 'data', 'wrfout_d01_2011-01-24_01:00:00')

# Read in the wrfout file using the netCDF4.Dataset method (I think you can also do this with an xarray method)
netcdf_data = netCDF4.Dataset(datapath)
netcdf_data

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format NETCDF3):
    TITLE:  OUTPUT FROM WRF V3.8.1 MODEL
    START_DATE: 2011-01-24_00:00:00
    SIMULATION_START_DATE: 2011-01-23_12:00:00
    WEST-EAST_GRID_DIMENSION: 192
    SOUTH-NORTH_GRID_DIMENSION: 192
    BOTTOM-TOP_GRID_DIMENSION: 36
    DX: 12000.0
    DY: 12000.0
    SKEBS_ON: 0
    SPEC_BDY_FINAL_MU: 1
    USE_Q_DIABATIC: 0
    GRIDTYPE: C
    DIFF_OPT: 1
    KM_OPT: 4
    DAMP_OPT: 0
    DAMPCOEF: 0.2
    KHDIF: 0.0
    KVDIF: 0.0
    MP_PHYSICS: 10
    RA_LW_PHYSICS: 1
    RA_SW_PHYSICS: 1
    SF_SFCLAY_PHYSICS: 2
    SF_SURFACE_PHYSICS: 2
    BL_PBL_PHYSICS: 5
    CU_PHYSICS: 3
    SF_LAKE_PHYSICS: 0
    SURFACE_INPUT_SOURCE: 1
    SST_UPDATE: 0
    GRID_FDDA: 1
    GFDDA_INTERVAL_M: 180
    GFDDA_END_H: 132
    GRID_SFDDA: 1
    SGFDDA_INTERVAL_M: 360
    SGFDDA_END_H: 120
    HYPSOMETRIC_OPT: 2
    USE_THETA_M: 0
    SF_URBAN_PHYSICS: 0
    SHCU_PHYSICS: 0
    MFSHCONV: 0
    FEEDBACK: 1


In [5]:
# Create an xarray.Dataset from the wrf qurery_variables.
query_variables = [
    'T2',
    'CLDFRA',
    'COSZEN',
    'SWDDNI',
    'SWDDIF',
    'height_agl',
    'wspd',
    'wdir',
    'rh',
    'pressure',
    'geopotential',
]

met_data = _wrf2xarray(netcdf_data, query_variables)

ValueError: cannot rename 'geopotential' because it is not a variable or dimension in this dataset

In [8]:
met_data

<xarray.Dataset>
Dimensions:     (Time: 24, bottom_top: 35, south_north: 191, west_east: 191)
Coordinates:
    XLONG       (south_north, west_east) float32 -95.43 -95.31 ... -64.27 -64.12
    XLAT        (south_north, west_east) float32 30.95 30.95 ... 47.55 47.51
    XTIME       (Time) float32 780.0 840.0 900.0 ... 2.04e+03 2.1e+03 2.16e+03
  * Time        (Time) datetime64[ns] 2011-01-24T01:00:00 ... 2011-01-25
Dimensions without coordinates: bottom_top, south_north, west_east
Data variables:
    T2          (Time, south_north, west_east) float32 287.6 287.6 ... 264.0
    CLDFRA      (Time, bottom_top, south_north, west_east) float32 0.0 ... 0.0
    COSZEN      (Time, south_north, west_east) float32 -0.2395 ... -0.4718
    SWDDNI      (Time, south_north, west_east) float32 0.0 0.0 0.0 ... 0.0 0.0
    SWDDIF      (Time, south_north, west_east) float32 0.0 0.0 0.0 ... 0.0 0.0
    height_agl  (Time, bottom_top, south_north, west_east) float32 10.12 ... ...
    wspd        (Time, bottom_top, south_north, west_east) float32 1.083 ... ...
    wdir        (Time, bottom_top, south_north, west_east) float32 195.7 ... ...
    rh          (Time, bottom_top, south_north, west_east) float32 57.57 ... ...
    pressure    (Time, bottom_top, south_north, west_east) float32 1.009e+03 ...
    geopt       (Time, bottom_top, south_north, west_east) float32 697.8 ... ...
Attributes:
    TITLE:                            OUTPUT FROM WRF V3.8.1 MODEL
    START_DATE:                      2011-01-24_00:00:00
    SIMULATION_START_DATE:           2011-01-23_12:00:00
    WEST-EAST_GRID_DIMENSION:        192
    SOUTH-NORTH_GRID_DIMENSION:      192
    BOTTOM-TOP_GRID_DIMENSION:       36
    DX:                              12000.0
    DY:                              12000.0
    SKEBS_ON:                        0
    SPEC_BDY_FINAL_MU:               1
    USE_Q_DIABATIC:                  0
    GRIDTYPE:                        C
    DIFF_OPT:                        1
    KM_OPT:                          4
    DAMP_OPT:                        0
    DAMPCOEF:                        0.2
    KHDIF:                           0.0
    KVDIF:                           0.0
    MP_PHYSICS:                      10
    RA_LW_PHYSICS:                   1
    RA_SW_PHYSICS:                   1
    SF_SFCLAY_PHYSICS:               2
    SF_SURFACE_PHYSICS:              2
    BL_PBL_PHYSICS:                  5
    CU_PHYSICS:                      3
    SF_LAKE_PHYSICS:                 0
    SURFACE_INPUT_SOURCE:            1
    SST_UPDATE:                      0
    GRID_FDDA:                       1
    GFDDA_INTERVAL_M:                180
    GFDDA_END_H:                     132
    GRID_SFDDA:                      1
    SGFDDA_INTERVAL_M:               360
    SGFDDA_END_H:                    120
    HYPSOMETRIC_OPT:                 2
    USE_THETA_M:                     0
    SF_URBAN_PHYSICS:                0
    SHCU_PHYSICS:                    0
    MFSHCONV:                        0
    FEEDBACK:                        1
    SMOOTH_OPTION:                   0
    SWRAD_SCAT:                      1.0
    W_DAMPING:                       0
    DT:                              45.0
    RADT:                            15.0
    BLDT:                            0.0
    CUDT:                            0.0
    AER_OPT:                         0
    SWINT_OPT:                       1
    AER_TYPE:                        1
    AER_AOD550_OPT:                  1
    AER_ANGEXP_OPT:                  1
    AER_SSA_OPT:                     1
    AER_ASY_OPT:                     1
    AER_AOD550_VAL:                  0.12
    AER_ANGEXP_VAL:                  1.3
    AER_SSA_VAL:                     1e-45
    AER_ASY_VAL:                     1e-45
    MOIST_ADV_OPT:                   1
    SCALAR_ADV_OPT:                  1
    TKE_ADV_OPT:                     1
    DIFF_6TH_OPT:                    0
    DIFF_6TH_FACTOR:                 0.12
    FGDT:                       

In [7]:
variables = {
    'XLAT': 'XLAT',
    'XLONG': 'XLONG',
    'T2': 'temp_air',
    'CLDFRA': 'cloud_fraction',
    'COSZEN': 'cos_zenith',
    'SWDDNI': 'dni',
    'SWDDIF': 'dhi',
    'height_agl': 'height_agl',
    'wspd': 'wspd',
    'wdir': 'wdir',
    'rh': 'rel_humidity',
    'pressure': 'pressure',
    'geopot': 'geopotential',
}

# Rename the variables
met_data = xr.Dataset.rename(met_data, variables)
met_data

ValueError: cannot rename 'geopot' because it is not a variable or dimension in this dataset